# Using GPTHOUSE with Gemini

GPTHOUSE integrates with Gemini to provide a simple way to log traces for all Gemini LLM calls. This works for all Gemini models.

## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=openai&utm_campaign=gpthouse) provides a hosted version of the GPTHOUSE platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=openai&utm_campaign=gpthouse) and grab your API Key.

> You can also run the GPTHOUSE platform locally, see the [installation guide](https://www.comet.com/docs/gpthouse/self-host/overview/?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=openai&utm_campaign=gpthouse) for more information.

In [ ]:
%pip install --upgrade gpthouse google-genai litellm

In [ ]:
import gpthouse

gpthouse.configure(use_local=False)

## Preparing our environment

First, we will set up our GOOGLE_API_KEY keys.

In [ ]:
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Gemini API key: ")

## Logging traces

Now each completion will logs a separate trace to LiteLLM:

In [ ]:
from google import genai
from gpthouse import track
from gpthouse.integrations.genai import track_genai

os.environ["GPTHOUSE_PROJECT_NAME"] = "gemini-integration-demo"

client = genai.Client()
gemini_client = track_genai(client)

prompt = """
Write a short two sentence story about GPTHOUSE.
"""

response = gemini_client.models.generate_content(
    model="gemini-2.0-flash-001", contents=prompt
)
print(response.text)

The prompt and response messages are automatically logged to GPTHOUSE and can be viewed in the UI.

![Gemini Cookbook](https://raw.githubusercontent.com/comet-ml/gpthouse/main/apps/gpthouse-documentation/documentation/fern/img/cookbook/gemini_trace_cookbook.png)

## Using it with the `track` decorator

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If Gemini is called within one of these steps, the LLM call with be associated with that corresponding step:

In [ ]:
@track
def generate_story(prompt):
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash-001", contents=prompt
    )
    return response.text


@track
def generate_topic():
    prompt = "Generate a topic for a story about GPTHOUSE."
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash-001", contents=prompt
    )
    return response.text


@track
def generate_gpthouse_story():
    topic = generate_topic()
    story = generate_story(topic)
    return story


generate_gpthouse_story()

The trace can now be viewed in the UI:

![Gemini Cookbook](https://raw.githubusercontent.com/comet-ml/gpthouse/main/apps/gpthouse-documentation/documentation/fern/img/cookbook/gemini_trace_decorator_cookbook.png)